In [1]:
import glob
import re
import hipscat_import.pipeline as runner
from hipscat_import.index.arguments import IndexArguments
import numpy as np
import dask
import dask.dataframe as dd
import pyarrow.parquet as pq
import pyarrow as pa
from tqdm import tqdm
import pyarrow.parquet as pq
from astropy.io import ascii
import pandas as pd
import dask.dataframe as dd
import dask

dask.config.set({"dataframe.convert-string": False})


In [2]:
from dask.distributed import Client

client = Client(
        local_directory="/data3/epyc/data3/hipscat/tmp/",
        n_workers=12,
        threads_per_worker=1,
    )
client

/astro/users/mmd11/.conda/envs/hipscatenv/lib/python3.10/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 33572 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:33572/status,
Dashboard: http://127.0.0.1:33572/status,Workers: 12
Total threads: 12,Total memory: 0.98 TiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:34314,Workers: 12
Dashboard: http://127.0.0.1:33572/status,Total threads: 12
Started: Just now,Total memory: 0.98 TiB
Comm: tcp://127.0.0.1:40922,Total threads: 1
Dashboard: http://127.0.0.1:43730/status,Memory: 83.97 GiB
Nanny: tcp://127.0.0.1:35925,


In [3]:
single_file = "/data3/epyc/data3/hipscat/raw/gaia/epoch_parquet/epoch_data_786097-786431.parquet"

parquet_file = pq.ParquetFile(single_file)
schema = parquet_file.schema.to_arrow_schema()
schema = schema.insert(0, pa.field("_hipscat_index", pa.uint64()))

In [4]:
left_table = dd.read_parquet(
        path="/data3/epyc/data3/hipscat/catalogs/gaia_dr3/gaia_source_id_hc_index",
        engine="pyarrow",
        filesystem="arrow",
    )

In [ ]:
files = glob.glob("/data3/epyc/data3/hipscat/raw/gaia/epoch_photometry/Epoch**")

for file in tqdm(files):
    match = re.match(r".*EpochPhotometry_([\d]+-[\d]+).csv.gz", str(file))
    file_id = match.group(1)
    
    astropy_table = ascii.read(file, format='ecsv')
    
    right_table = dd.from_pandas(astropy_table.to_pandas(), chunksize=2_000_000_000)
    result = left_table.merge(right_table, how="right", left_index=True, right_on="source_id")
    result = result.repartition(npartitions=1)

    result.to_parquet(f"/data3/epyc/data3/hipscat/raw/gaia/epoch_w_hipscat/bulk-{file_id}",
                      engine="pyarrow",
                      schema=schema,
                      write_index=False)


  0%|          | 0/3386 [00:00<?, ?it/s]/astro/users/mmd11/.conda/envs/hipscatenv/lib/python3.10/site-packages/distributed/client.py:3162: UserWarning: Sending large graph of size 22.71 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
  0%|          | 1/3386 [03:13<182:21:07, 193.93s/it]/astro/users/mmd11/.conda/envs/hipscatenv/lib/python3.10/site-packages/distributed/client.py:3162: UserWarning: Sending large graph of size 27.88 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
  0%|          | 2/3386 [06:19<177:24:26, 188.73s/it]/astro/users/mmd11/.conda/envs/hipscatenv/lib/python3.10/site-packages/distributed/client.py:3162: UserWarning: Sending large graph of size 27.52 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
  0%|          | 3/3386 [09:16<172:41:14, 183.76s/it]/astro/users/mmd11/.conda/envs/hi

  1%|          | 27/3386 [1:25:42<183:05:17, 196.22s/it]/astro/users/mmd11/.conda/envs/hipscatenv/lib/python3.10/site-packages/distributed/client.py:3162: UserWarning: Sending large graph of size 45.83 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
  1%|          | 28/3386 [1:29:01<183:52:02, 197.12s/it]/astro/users/mmd11/.conda/envs/hipscatenv/lib/python3.10/site-packages/distributed/client.py:3162: UserWarning: Sending large graph of size 35.79 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
  1%|          | 29/3386 [1:32:08<180:55:57, 194.03s/it]/astro/users/mmd11/.conda/envs/hipscatenv/lib/python3.10/site-packages/distributed/client.py:3162: UserWarning: Sending large graph of size 32.30 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
  1%|          | 30/3386 [1:35:15<178:50:50, 191.85s/it]/astro/u

  2%|▏         | 54/3386 [2:41:42<152:13:48, 164.47s/it]/astro/users/mmd11/.conda/envs/hipscatenv/lib/python3.10/site-packages/distributed/client.py:3162: UserWarning: Sending large graph of size 31.24 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
  2%|▏         | 55/3386 [2:44:27<152:28:02, 164.78s/it]/astro/users/mmd11/.conda/envs/hipscatenv/lib/python3.10/site-packages/distributed/client.py:3162: UserWarning: Sending large graph of size 32.81 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
  2%|▏         | 56/3386 [2:47:12<152:24:37, 164.77s/it]/astro/users/mmd11/.conda/envs/hipscatenv/lib/python3.10/site-packages/distributed/client.py:3162: UserWarning: Sending large graph of size 23.33 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
  2%|▏         | 57/3386 [2:49:52<151:11:38, 163.50s/it]/astro/u

2024-03-27 20:59:45,240 - distributed.worker - ERROR - Worker stream died during communication: tcp://127.0.0.1:41241
Traceback (most recent call last):
  File "/astro/users/mmd11/.conda/envs/hipscatenv/lib/python3.10/site-packages/tornado/iostream.py", line 861, in _read_to_buffer
    bytes_read = self.read_from_fd(buf)
  File "/astro/users/mmd11/.conda/envs/hipscatenv/lib/python3.10/site-packages/tornado/iostream.py", line 1116, in read_from_fd
    return self.socket.recv_into(buf, len(buf))
ConnectionResetError: [Errno 104] Connection reset by peer

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/astro/users/mmd11/.conda/envs/hipscatenv/lib/python3.10/site-packages/distributed/worker.py", line 2056, in gather_dep
    response = await get_data_from_worker(
  File "/astro/users/mmd11/.conda/envs/hipscatenv/lib/python3.10/site-packages/distributed/worker.py", line 2860, in get_data_from_worker
    response = await send_r

2024-03-27 21:01:19,210 - distributed.worker - ERROR - failed during get data with tcp://127.0.0.1:40300 -> tcp://127.0.0.1:42649
Traceback (most recent call last):
  File "/astro/users/mmd11/.conda/envs/hipscatenv/lib/python3.10/site-packages/tornado/iostream.py", line 861, in _read_to_buffer
    bytes_read = self.read_from_fd(buf)
  File "/astro/users/mmd11/.conda/envs/hipscatenv/lib/python3.10/site-packages/tornado/iostream.py", line 1116, in read_from_fd
    return self.socket.recv_into(buf, len(buf))
TimeoutError: [Errno 110] Connection timed out

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/astro/users/mmd11/.conda/envs/hipscatenv/lib/python3.10/site-packages/distributed/worker.py", line 1780, in get_data
    response = await comm.read(deserializers=serializers)
  File "/astro/users/mmd11/.conda/envs/hipscatenv/lib/python3.10/site-packages/distributed/comm/tcp.py", line 237, in read
    convert_stream_closed_err

...

 73%|███████▎  | 2470/3386 [121:28:16<40:44:30, 160.12s/it]/astro/users/mmd11/.conda/envs/hipscatenv/lib/python3.10/site-packages/distributed/client.py:3162: UserWarning: Sending large graph of size 19.21 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
 73%|███████▎  | 2471/3386 [121:30:57<40:46:33, 160.43s/it]/astro/users/mmd11/.conda/envs/hipscatenv/lib/python3.10/site-packages/distributed/client.py:3162: UserWarning: Sending large graph of size 17.86 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
 73%|███████▎  | 2472/3386 [121:33:34<40:26:34, 159.29s/it]/astro/users/mmd11/.conda/envs/hipscatenv/lib/python3.10/site-packages/distributed/client.py:3162: UserWarning: Sending large graph of size 17.11 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
 73%|███████▎  | 2473/3386 [121:36:09<40:04:14, 158.00s

 74%|███████▍  | 2522/3386 [123:46:55<38:47:02, 161.60s/it]/astro/users/mmd11/.conda/envs/hipscatenv/lib/python3.10/site-packages/distributed/client.py:3162: UserWarning: Sending large graph of size 30.87 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
 75%|███████▍  | 2523/3386 [123:49:39<38:55:06, 162.35s/it]/astro/users/mmd11/.conda/envs/hipscatenv/lib/python3.10/site-packages/distributed/client.py:3162: UserWarning: Sending large graph of size 28.28 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
 75%|███████▍  | 2524/3386 [123:52:23<39:00:44, 162.93s/it]/astro/users/mmd11/.conda/envs/hipscatenv/lib/python3.10/site-packages/distributed/client.py:3162: UserWarning: Sending large graph of size 38.65 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
 75%|███████▍  | 2525/3386 [123:55:15<39:35:47, 165.56s

 76%|███████▌  | 2574/3386 [126:09:41<35:19:29, 156.61s/it]/astro/users/mmd11/.conda/envs/hipscatenv/lib/python3.10/site-packages/distributed/client.py:3162: UserWarning: Sending large graph of size 20.47 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
 76%|███████▌  | 2575/3386 [126:12:19<35:22:09, 157.00s/it]/astro/users/mmd11/.conda/envs/hipscatenv/lib/python3.10/site-packages/distributed/client.py:3162: UserWarning: Sending large graph of size 17.13 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
 76%|███████▌  | 2576/3386 [126:14:58<35:30:57, 157.85s/it]/astro/users/mmd11/.conda/envs/hipscatenv/lib/python3.10/site-packages/distributed/client.py:3162: UserWarning: Sending large graph of size 14.53 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
 76%|███████▌  | 2577/3386 [126:17:34<35:19:45, 157.21s

 78%|███████▊  | 2626/3386 [128:30:47<32:53:31, 155.80s/it]/astro/users/mmd11/.conda/envs/hipscatenv/lib/python3.10/site-packages/distributed/client.py:3162: UserWarning: Sending large graph of size 14.02 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
 78%|███████▊  | 2627/3386 [128:33:21<32:43:08, 155.19s/it]/astro/users/mmd11/.conda/envs/hipscatenv/lib/python3.10/site-packages/distributed/client.py:3162: UserWarning: Sending large graph of size 13.34 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
 78%|███████▊  | 2628/3386 [128:35:53<32:29:07, 154.28s/it]/astro/users/mmd11/.conda/envs/hipscatenv/lib/python3.10/site-packages/distributed/client.py:3162: UserWarning: Sending large graph of size 13.40 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
 78%|███████▊  | 2629/3386 [128:38:29<32:32:27, 154.75s

 81%|████████  | 2748/3386 [133:50:33<28:14:35, 159.37s/it]/astro/users/mmd11/.conda/envs/hipscatenv/lib/python3.10/site-packages/distributed/client.py:3162: UserWarning: Sending large graph of size 20.20 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
 81%|████████  | 2749/3386 [133:53:10<28:04:17, 158.65s/it]/astro/users/mmd11/.conda/envs/hipscatenv/lib/python3.10/site-packages/distributed/client.py:3162: UserWarning: Sending large graph of size 36.25 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
 81%|████████  | 2750/3386 [133:55:57<28:26:51, 161.02s/it]/astro/users/mmd11/.conda/envs/hipscatenv/lib/python3.10/site-packages/distributed/client.py:3162: UserWarning: Sending large graph of size 40.49 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
 81%|████████  | 2751/3386 [133:58:45<28:48:06, 163.29s

In [11]:
files = glob.glob("/data3/epyc/data3/hipscat/raw/gaia/epoch_w_hipscat/bulk-*/*.parquet")

from hipscat_import.pipeline import ImportArguments

args = ImportArguments(
    input_file_list=files,
    file_reader="parquet",
    use_hipscat_index=True,
    lowest_healpix_order=2,
    output_artifact_name="epoch_photometry",
    output_path="/data3/epyc/data3/hipscat/catalogs/",
)

Planning  :   0%|          | 0/5 [00:00<?, ?it/s]

In [12]:
%%time

from hipscat_import.pipeline import pipeline_with_client
pipeline_with_client(args, client)

Mapping   :   0%|          | 0/3386 [00:00<?, ?it/s]

Binning   :   0%|          | 0/2 [00:00<?, ?it/s]

Splitting :   0%|          | 0/3386 [00:00<?, ?it/s]

Reducing  :   0%|          | 0/198 [00:00<?, ?it/s]

Finishing :   0%|          | 0/5 [00:00<?, ?it/s]

CPU times: user 3min 22s, sys: 1min 13s, total: 4min 36s
Wall time: 12min 38s


In [32]:
import hipscat
from hipscat.inspection import plot_pixels, plot_points

cat = hipscat.read_from_hipscat("/data3/epyc/data3/hipscat/catalogs/tic")
plot_points(cat)
plot_pixels(cat)

FileNotFoundError: [Errno 2] No such file or directory: '/data3/epyc/data3/hipscat/catalogs/tic/catalog_info.json'

In [17]:
from dask.distributed import Client, performance_report

print(file)

with performance_report(filename="./reports/epoch_convert_report.html"):
    match = re.match(r".*EpochPhotometry_([\d]+-[\d]+).csv.gz", str(file))
    file_id = match.group(1)
    
    astropy_table = ascii.read(file, format='ecsv')
    
    right_table = dd.from_pandas(astropy_table.to_pandas(), chunksize=2_000_000_000)
    result = left_table.merge(right_table, how="right", left_index=True, right_on="source_id")
    result = result.repartition(npartitions=1)

    result.to_parquet(f"./reports/bulk-{file_id}",
                      engine="pyarrow",
                      schema=schema,
                      write_index=False)

/data3/epyc/data3/hipscat/raw/gaia/epoch_photometry/EpochPhotometry_786097-786431.csv.gz


/astro/users/mmd11/.conda/envs/hipscatenv/lib/python3.10/site-packages/distributed/client.py:3162: UserWarning: Sending large graph of size 17.12 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


In [20]:
from IPython.display import HTML

HTML("./reports/epoch_convert_report.html")

In [19]:
!pip install ipywidgets

Defaulting to user installation because normal site-packages is not writeable


In [21]:
import cProfile

profiler = cProfile.Profile()
profiler.enable()

astropy_table = ascii.read(file, format='ecsv')

astropy_table.to_pandas().to_parquet("./reports/temp.parquet")

profiler.disable()
profiler.dump_stats("./reports/cprofile.thing")

In [29]:
%%time

file = "/data3/epyc/data3/hipscat/raw/gaia/epoch_photometry/EpochPhotometry_731376-731725.csv.gz"

astropy_table = ascii.read(file, format='ecsv')
print(len(astropy_table))
astropy_table.to_pandas().to_parquet("./reports/temp.parquet")

4505
CPU times: user 12.8 s, sys: 1.84 s, total: 14.7 s
Wall time: 14 s


In [31]:
%%time

file = "/data3/epyc/data3/hipscat/raw/gaia/epoch_photometry/EpochPhotometry_731376-731725.csv.gz"

pandas_table = pd.read_csv(file, comment="#")
print(len(pandas_table))
pandas_table.to_parquet("./reports/temp.parquet")

4505
CPU times: user 6.02 s, sys: 1.21 s, total: 7.23 s
Wall time: 6.81 s
